In [1]:
import sys
sys.path.insert(1, '../')  # to load from any submodule in the repo

import utils.readOBJ as reader
import utils.dpcr_utils as utils
import utils.dpcr_generator as generator

import models3D.neighbor_predictor3D as neighbor_predictor3D
import models3D.detector3D as detector3D

import torch
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import time
from mpl_toolkits.mplot3d import Axes3D
import itertools
import _pickle as cPickle

device = torch.device("cpu")

if torch.device("cuda"):
    device = torch.device("cuda")
    print("Using", torch.cuda.device_count(), "CUDA devices")
    
torch.no_grad()

seed_file = open('../utils/seed.txt', "r")
seed = int(seed_file.read())
seed_file.close()

print ("Using Seed:", seed)
    
torch.manual_seed(seed)
np.random.seed(seed)

Using 1 CUDA devices
Using Seed: 34897567


In [2]:
predictor = neighbor_predictor3D.Model(k = 20, emb_dims = 1024, dropout = 0.5, neighbors = 6).to(device)
predictor = torch.nn.DataParallel(predictor)

#detector = detector3D.Model(k = 10, emb_dims = 1024, dropout = 0.5).to(device)
#detector = torch.nn.DataParallel(detector)

predictor_checkpoint = torch.load('../models3D/model_predictor_3d_batched_0624_e50.t7')
#detector_checkpoint = torch.load('../models3D/model_detector_3d_batched_e50.t7')

predictor.load_state_dict(predictor_checkpoint['model_state_dict'][-1])
print ("> Loaded predictor model (%d epochs)" % len(predictor_checkpoint['train_time']))

#detector.load_state_dict(detector_checkpoint['model_state_dict'])
#print ("> Loaded detector model (%d epochs)" % detector_checkpoint['epoch'])

_ = predictor.eval()
#_ = detector.eval()

> Loaded predictor model (50 epochs)


In [3]:
train_data = None
with open('../data/train_test_data_new/train_data_test', 'rb') as file:
    train_data = cPickle.load(file)

In [6]:
neighbors_count = 6

# extract train- & test data (and move to device) --------------------------------------------

pts = train_data["pts"].float()
knn = train_data["knn"]

#train_bins = train_data["train_bins"]
test_samples = train_data["test_samples"]

In [26]:
start = time.time()

model, _ = reader.readOBJ("../data/models/faces/clean/face_%02d_clean.obj" % (12))
    
model = torch.from_numpy(model).to(device)

# mean centering
model = model - torch.mean(model, dim = 0)

# scaling to fit 2x2x2 bounding box
min_vals, _ = torch.min(model, dim = 0)
max_vals, _ = torch.max(model, dim = 0)
scale = torch.max(torch.abs(min_vals),  torch.abs(max_vals))
model = model / scale

knn = utils.knn(model, 6)
pts = model.float()

test_samples = generator.getData(pts, 20)

print ("Generated %d Test Models" % len(test_samples))
print ("Total Time:", time.time() - start)

Generated 157 Test Models
Total Time: 5.144001722335815


In [31]:
models = []
test_samples = []
knn = []

start_id = 0

for i in range(11,13):
    
    print ("Processing model %d/%d.." % (i, 10))
    
    start = time.time()
    
    model, _ = reader.readOBJ("../data/models/faces/clean/face_%02d_clean.obj" % (i))
    
    model = torch.from_numpy(model).to(device)
    
    # mean centering
    model = model - torch.mean(model, dim = 0)
    
    # scaling to fit 2x2x2 bounding box
    min_vals, _ = torch.min(model, dim = 0)
    max_vals, _ = torch.max(model, dim = 0)
    scale = torch.max(torch.abs(min_vals),  torch.abs(max_vals))
    model = model / scale
    
    models.append(model)
    
    knn.append(utils.knn(model, 6) + start_id)
    
    for sample in generator.getData(model, 10):
    
        sample[:, 0] += start_id
        test_samples.append(sample)
    
    print ("   ..done! (%.1fs)" % (time.time() - start))
    
    start_id += model.size(0)
    
    
print ("\n#Test Samples: %d" % len(test_samples))

pts = torch.cat(models).float()
knn = torch.cat(knn).long()

Processing model 11/10..
   ..done! (3.4s)
Processing model 12/10..
   ..done! (3.3s)

#Test Samples: 139


In [32]:
neighbors_count = 6

# pre-permute the neighbors (labels) ----------------------------------------------------------

print ("\nPre-Computing Neighbor Permutations..")

start = time.time()

neighbors_dirs = pts[knn] - pts[:, None, :]

perms_np = np.array(list(itertools.permutations(np.arange(neighbors_count))))
perms = torch.from_numpy(perms_np).long().to(device)

neighbors_dirs_perms = torch.zeros((neighbors_dirs.size(0), perms.size(0), neighbors_count, 3))

for (i, p) in enumerate(perms):
    neighbors_dirs_perms[:,i] = neighbors_dirs[:, p, :]

end = time.time()

print ("  > done! (%.2fs)" % (end - start))


# set up loss and batch size -------------------------------------------------------------------

loss_function = torch.nn.MSELoss(reduction='sum')


Pre-Computing Neighbor Permutations..
  > done! (0.95s)


In [35]:
validation_loss = len(predictor_checkpoint['model_state_dict']) * [0]

print (validation_loss)

for i in range(len(predictor_checkpoint['model_state_dict'])):
    
    predictor = neighbor_predictor3D.Model(k = 20, emb_dims = 1024, dropout = 0.5, neighbors = 6).to(device)
    predictor = torch.nn.DataParallel(predictor)

    predictor.load_state_dict(predictor_checkpoint['model_state_dict'][i])
    print ("> Loaded predictor model (%d epochs)" % (i+1))

    _ = predictor.eval()

    cum_test_loss = 0
    cum_test_loss_per_point = 0
    loss_function = torch.nn.MSELoss(reduction='sum')

    for sample in test_samples:

        sample_ids = sample[:,0].flatten()

        input_tensor = pts[sample_ids].unsqueeze(0).transpose(1,2).to(device)

        p = predictor(input_tensor)  # size: (1, 3*NC, N)
        p = p.squeeze(0)                                        # size: (3*NC, N)
        p = p.transpose(0,1)                                    # size: (N, 3*NC)
        p = p.reshape((-1, 6, 3))                 # size: (N, NC, 3)

        t_perm = neighbors_dirs_perms[sample_ids].to(device)                 # size: (N, perm_count, NC, 3)

        # for each vertex compute a permutation of neighbors s.t. the sum of differences to the prediction is minimized
        minmatches = utils.matchPoints(p.detach(), y=None, perms=perms, y_perms=t_perm)  # size: (N, NC)

        # expand the permutation index
        gather_idx = minmatches.expand(3,minmatches.size(0),minmatches.size(1)).transpose(0,1).transpose(1,2)  # size: (N, NC, 3)

        # gather neighbor directions according to the permutations
        t = neighbors_dirs[sample_ids].gather(1, gather_idx).to(device)    # size: (N, NC, 3)

        loss = loss_function(p, t).item()
        loss_per_point = loss / sample_ids.numel()

        #print ("\nsample loss:", loss)
        #print ("sample loss_per_point:", loss_per_point)

        cum_test_loss += loss
        cum_test_loss_per_point += loss_per_point
        
    validation_loss[i] = cum_test_loss
    
    print ("\nEpoch %d:" % (i+1))
    print ("Average Validation Loss: %.3f" % (cum_test_loss / len(test_samples)))
    print ("Average Validation Loss (per point): %.3f" % (cum_test_loss_per_point / len(test_samples)))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
> Loaded predictor model (1 epochs)

Epoch 1:
Average Validation Loss: 16.553
Average Validation Loss (per point): 0.003
> Loaded predictor model (2 epochs)

Epoch 2:
Average Validation Loss: 16.553
Average Validation Loss (per point): 0.003
> Loaded predictor model (3 epochs)

Epoch 3:
Average Validation Loss: 16.553
Average Validation Loss (per point): 0.003
> Loaded predictor model (4 epochs)


KeyboardInterrupt: 

In [38]:
a = [1,2,3]

import copy

b = copy.deepcopy(a)

In [39]:
b

[1, 2, 3]